### 1. Install and Import Required Libraries

In [3]:
!pip install msal requests pandas python-dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)


In [18]:
import msal
import requests
import json
import os
import pandas as pd
from dotenv import load_dotenv

# Find the .env file by searching up the directory tree
# This makes it independent of where the notebook is run from
current_dir = os.getcwd()
# Go up one level since the notebook is in the 'Copilot' folder
project_root = os.path.dirname(current_dir) 
dotenv_path = os.path.join(project_root, '.env')

# Check if the .env file exists at the constructed path
if not os.path.exists(dotenv_path):
    # If not found, assume the CWD is the project root
    dotenv_path = os.path.join(os.getcwd(), '.env')

load_dotenv(dotenv_path=dotenv_path)

True

### 2. Define Entra App and API Configuration

This notebook loads credentials from your root `.env` file. Make sure it contains the following variables:
- `CopilotReportAPIPythonClient_Id`
- `CopilotReportAPIPythonClient_Secret`
- `CopilotAPIPythonClient_Tenant`

**Note on Permissions:** This notebook uses **Application** permissions (client credentials flow). In your Entra App Registration, grant the following permissions under **Application permissions** for Microsoft Graph:
- `Reports.Read.All`

**Important:** 
- After adding the application permission, you must click **Grant admin consent** for the tenant.
- The app uses a client secret for authentication (no user interaction required).

**Configuration Steps:**
1. In your App Registration, go to the **API permissions** tab.
2. Click **Add a permission** > **Microsoft Graph** > **Application permissions**.
3. Select `Reports.Read.All` and click **Add permissions**.
4. Click **Grant admin consent for [your tenant]** and confirm.
5. In the **Certificates & secrets** tab, create a new client secret and save it to your `.env` file.

In [19]:
# Load Entra App Registration details from .env file
client_id = os.getenv("CopilotReportAPIPythonClient_Id")
client_secret = os.getenv("CopilotReportAPIPythonClient_Secret")
tenant_id = os.getenv("CopilotAPIPythonClient_Tenant")
authority = f"https://login.microsoftonline.com/{tenant_id}"

# API details for Copilot Usage Reports
# Using the Reports.Read.All application permission scope
# For application permissions, use /.default scope
scopes = ["https://graph.microsoft.com/.default"]

# API endpoint for getting user detail reports
base_api_endpoint = "https://graph.microsoft.com/v1.0/copilot/reports"

### 3. Acquire Access Token

This step uses the MSAL library to acquire an access token using the client credentials flow (application permissions). This method authenticates the application itself using the client secret, without requiring user interaction.

In [20]:
# Create a confidential client application for client credentials flow
app = msal.ConfidentialClientApplication(
    client_id,
    authority=authority,
    client_credential=client_secret
)

# Acquire token using client credentials (application permissions)
result = app.acquire_token_for_client(scopes=scopes)

if "access_token" in result:
    access_token = result["access_token"]
    print("Access token acquired successfully using client credentials.")
else:
    print("Could not acquire access token.")
    print(result.get("error"))
    print(result.get("error_description"))
    access_token = None

Access token acquired successfully using client credentials.


### 4. Call the Copilot Usage User Detail Report API

Now we will use the acquired access token to make a `GET` request to retrieve Microsoft 365 Copilot usage data by user.

**Available Report Periods:**
- `D7` - Last 7 days
- `D30` - Last 30 days
- `D90` - Last 90 days
- `D180` - Last 180 days
- `ALL` - All available periods

The report includes user activity data such as:
- Last activity dates for various Copilot applications (Teams, Word, Excel, PowerPoint, Outlook, OneNote, Loop)
- User principal name (anonymized in reports)
- Display name (anonymized in reports)

In [21]:
if access_token:
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    # Set the report period (D7, D30, D90, D180, or ALL)
    report_period = "D90"  # Change this to your desired period
    
    # Construct the API endpoint with period parameter
    # Note: This API returns CSV format, not JSON
    api_endpoint = f"{base_api_endpoint}/getMicrosoft365CopilotUsageUserDetail(period='{report_period}')"

    try:
        response = requests.get(api_endpoint, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        
        # The response is in CSV format, so we'll parse it with pandas
        from io import StringIO
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data)
        
        print(f"API call successful. Retrieved {len(df)} user records.")
        api_response = df  # Store DataFrame instead of JSON

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print(f"Response body: {response.text}")
        api_response = None
    except Exception as err:
        print(f"An error occurred: {err}")
        api_response = None
else:
    print("Cannot make API call without an access token.")
    api_response = None

API call successful. Retrieved 24 user records.


### 5. Display the API Response

Let's display the CSV data that was returned from the API and converted to a pandas DataFrame.

In [22]:
if api_response is not None and isinstance(api_response, pd.DataFrame):
    # Display basic information about the data
    print(f"DataFrame shape: {api_response.shape}")
    print(f"\nColumns: {list(api_response.columns)}")
    print(f"\nFirst few rows:")
    display(api_response.head())
else:
    print("No valid API response to display.")

DataFrame shape: (24, 13)

Columns: ['Report Refresh Date', 'User Principal Name', 'Display Name', 'Last Activity Date', 'Copilot Chat Last Activity Date', 'Microsoft Teams Copilot Last Activity Date', 'Word Copilot Last Activity Date', 'Excel Copilot Last Activity Date', 'PowerPoint Copilot Last Activity Date', 'Outlook Copilot Last Activity Date', 'OneNote Copilot Last Activity Date', 'Loop Copilot Last Activity Date', 'Report Period']

First few rows:


,Report Refresh Date,User Principal Name,Display Name,Last Activity Date,Copilot Chat Last Activity Date,Microsoft Teams Copilot Last Activity Date,Word Copilot Last Activity Date,Excel Copilot Last Activity Date,PowerPoint Copilot Last Activity Date,Outlook Copilot Last Activity Date,OneNote Copilot Last Activity Date,Loop Copilot Last Activity Date,Report Period
0,2025-12-15,DED3FB44B8F67994DC3EBBFBB6CBD69B,E6863A7F7BCE57E34DBFD2747CFB870E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90
1,2025-12-15,915204FEA5A74CE7A407C8C66DBC45F2,2CB7BC1B5DAA82034087A63EA5C52804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90
2,2025-12-15,736E4078E9503624CCDCE4608FCB72A2,4B838AC2275E9416BF640FEF12EFEF78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90
3,2025-12-15,61257461F5660297C2172DA4A69FC52B,3E6B3BF3810DBE885E5C6EC73E9E4E91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90
4,2025-12-15,624382E229367B718492044E2C2AA070,24337E1C6FC144E641CEEE4265DA5356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90


### 6. Convert to DataFrame for Analysis

Convert the response data into a pandas DataFrame for easier data manipulation and analysis.

In [23]:
if api_response is not None and isinstance(api_response, pd.DataFrame):
    # The data is already a DataFrame from the API call
    df = api_response
    
    # Show summary statistics
    print(f"\n--- Activity Summary ---")
    if not df.empty:
        # Count users with activity in each product
        activity_columns = [col for col in df.columns if 'Last Activity Date' in col or 'LastActivityDate' in col]
        for col in activity_columns:
            if col in df.columns:
                # Count non-empty values
                active_users = df[col].notna().sum()
                print(f"{col}: {active_users} users with activity")
else:
    print("No data available to create DataFrame.")


--- Activity Summary ---
Last Activity Date: 0 users with activity
Copilot Chat Last Activity Date: 0 users with activity
Microsoft Teams Copilot Last Activity Date: 0 users with activity
Word Copilot Last Activity Date: 0 users with activity
Excel Copilot Last Activity Date: 0 users with activity
PowerPoint Copilot Last Activity Date: 0 users with activity
Outlook Copilot Last Activity Date: 0 users with activity
OneNote Copilot Last Activity Date: 0 users with activity
Loop Copilot Last Activity Date: 0 users with activity


### 7. Export to CSV (Optional)

Save the usage data to a CSV file for further analysis or reporting.

In [24]:
if api_response is not None and isinstance(api_response, pd.DataFrame) and not api_response.empty:
    # Use the DataFrame from the API response
    df = api_response
    
    # Define output file path
    output_file = os.path.join(current_dir, f"copilot_usage_report_{report_period}.csv")
    
    # Save to CSV
    df.to_csv(output_file, index=False)
    print(f"Data exported to: {output_file}")
else:
    print("No data available to export.")

Data exported to: c:\Users\jhammonds\OneDrive - Microsoft\Documents\GitHub\MW-Toolbox\Copilot\copilot_usage_report_D90.csv


### 8. Additional Analysis Examples

Here are some examples of how you can analyze the data further.

In [ ]:
if api_response is not None and isinstance(api_response, pd.DataFrame) and not api_response.empty:
    df = api_response
    print("--- User Activity Analysis ---\n")
    
    # Count total active users (users with any last activity date)
    # Look for column that might contain overall activity date
    last_activity_cols = [col for col in df.columns if 'Last Activity Date' in col and 'Copilot' not in col]
    if last_activity_cols:
        total_active = df[last_activity_cols[0]].notna().sum()
        print(f"Total users with any Copilot activity: {total_active}")
    
    # Count users by specific product usage
    # Note: Column names in CSV may use spaces instead of camelCase
    product_column_patterns = [
        ('Copilot Chat', ['Copilot Chat Last Activity Date', 'copilotChatLastActivityDate']),
        ('Teams Copilot', ['Microsoft Teams Copilot Last Activity Date', 'microsoftTeamsCopilotLastActivityDate']),
        ('Word Copilot', ['Word Copilot Last Activity Date', 'wordCopilotLastActivityDate']),
        ('Excel Copilot', ['Excel Copilot Last Activity Date', 'excelCopilotLastActivityDate']),
        ('PowerPoint Copilot', ['PowerPoint Copilot Last Activity Date', 'powerPointCopilotLastActivityDate']),
        ('Outlook Copilot', ['Outlook Copilot Last Activity Date', 'outlookCopilotLastActivityDate']),
        ('OneNote Copilot', ['OneNote Copilot Last Activity Date', 'oneNoteCopilotLastActivityDate']),
        ('Loop Copilot', ['Loop Copilot Last Activity Date', 'loopCopilotLastActivityDate'])
    ]
    
    print("\n--- Activity by Product ---")
    for name, possible_cols in product_column_patterns:
        for col in possible_cols:
            if col in df.columns:
                # Count non-empty values (dates)
                active_count = df[col].apply(lambda x: str(x).strip() != '' and pd.notna(x)).sum()
                print(f"{name}: {active_count} users")
                break
    
    # Report refresh date
    refresh_date_cols = [col for col in df.columns if 'Report Refresh Date' in col or 'reportRefreshDate' in col]
    if refresh_date_cols and not df.empty:
        refresh_date = df[refresh_date_cols[0]].iloc[0]
        print(f"\nReport refresh date: {refresh_date}")
else:
    print("No data available for analysis.")